<font color = "gold"> <i> Code is read much more often than it is written, so plan accordingly </i> </font> 

https://devblogs.microsoft.com/oldnewthing/20070406-00/?p=27343 

espero que me salga :)

# <u> <font color = "bisque"> Tareas:</font> </u>
## <font color = "bisque"> * Detectar archivos con problemas de georrefercia </font> 
## <font color = "bisque"> * Implementar procesos de renombre </font> 

---

<b> <font color = "orangered"> MODO DE TRABAJO PROCESO DE RENOMBRE - v1 </font> </b>

Voy a trabajar sobre argendata/data que esté hasta ahora, en un fork propio.

<b> Sobre argendata/data lo que voy a hacer es tomar los .csv  originales y hacer lo siguiente (dependiendo el caso), generando .csv actualizados:</b>

- a los csv que tengan solo código geo (tipo iso3) agregarle columna name_long correspondiente a cada código según el geonomeclador. Además de fijarme si los códigos que tienen son coherentes al geonomeclador. Los que no los sean, los cambio

- a los csv que tienen un tipo de columna que refiere a georef (que no es geo_codigo) agrego columna correspondiente a lo que sea long_name del geonomeclador, y otra columna del geo_codigo 

- variaciones de lo anterior

<b> <font color = "orangered"> Entonces la columna original con data geo_ref NO LA TOCO, sumo columnas: geo_codigo + long_name del geonomenclador </font> </b>

<font color = "lightseagreen"> Los códigos/nombres que no estén considerados en el geonomeclador, si es que aparecen, los voy a documentar en un .csv del tipo: csv | topico | columna csv | problema | arreglo </font>

- Por ejemplo, si encuentro que una columna que sería de georef, y todo su contenido está en el geonomenclador salvo una fila, guardarlo como .csv con problema
- Por ejemplo si una columna es de georef que está en otro idioma y no tiene iso_3 (geo_codigo) 


<b> ¿Cómo considero si una columna es de georef en esta 1ra iteración? </b>
- considerar el input country_keys: nombres de columnas que refieren a algo geo referencial ➡️ https://github.com/argendatafundar/geonomencladores-interno/blob/main/cosas/country_keys.txt 
- 80% del contenido esa columna es parte de alguna de las columnas del geoneomenclador
- Por el momento solo código que esté en el geonomenclador en español

Observaciones: 
- PARA ESTE LABURO ARMÉ MI PROPIO BLOQUE DE ENCODERs, DELIMITERs
- Para esta versión de la tarea, en principio, solo consideraría contenido en español. 

<b> <font color = "orangered"> Productos: 
- .csv argendata con agregado de columnas para implementar renombres correspondientes
- .csv con lo que mencioné antes con .csv de argendata que presentarían problemas </font> </b>

Con lo implementado en mi fork de esta v1 y lo ya realizado, evaluamos próximos pasos a realizar.

---

#  <u> <font color = "orangered"> En cuanto a detectar archivos con probleas de georreferencia: </font> </u>

<font color = "gold"> Cada .csv de Argendata tiene su propio encodings y delimiters </font>. 

No todos los .csv que contempla Argendata tienen el mismo enconding y delimiter. Por eso antes de comenzar a trabajar voy a armar el siguiente input para la tarea: dataframe que contenga 4 columnas que sean 'TOPICO', 'archivo_csv', 'encoding' y delimiter'.

In [5]:
import os
import glob
import chardet
import csv
import pandas as pd

# ruta de tabajo donde estan los topicos con sus csv 
base_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata' #'/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata'

# lista para recolectar los datos
data = []

# tamanio del fragmento a leer (en este caso 100000 bytes)
sample_size = 100000

for topico in os.listdir(base_dir): # ejemplo SEBACO
    topico_path = os.path.join(base_dir, topico) #ejemplo /home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata/SEBACO

    # ignorar archivos sueltos como LICENSE y README.md
    if not os.path.isdir(topico_path):
        continue

    for archivo in os.listdir(topico_path):
        if archivo.endswith('.csv'):
            archivo_path = os.path.join(topico_path, archivo)

            # leer un fragmento del .csv para análisis
            with open(archivo_path, 'rb') as f:
                raw_sample = f.read(sample_size)

            # detectar encoding
            encoding_detected = 'utf-8'  # por defecto
            try:
                encoding_info = chardet.detect(raw_sample)
                if encoding_info and encoding_info['encoding']:
                    encoding_detected = encoding_info['encoding']
            except:
                pass

            # decodificar el fragmento
            try:
                sample = raw_sample.decode(encoding_detected, errors = 'replace')
            except:
                sample = raw_sample.decode('utf-8', errors = 'replace') 

            # detectar delimitador
            delimiter_detected = ','
            try:
                sniffer = csv.Sniffer()
                dialect = sniffer.sniff(sample)
                delimiter_detected = dialect.delimiter
            except:
                pass

            # agregar al resultado
            data.append({
                'TOPICO': topico,
                'archivo_csv': archivo,
                'encoding': encoding_detected,
                'delimiter': delimiter_detected
            })


df_resultado = pd.DataFrame(data)
df_codificacion = df_resultado

print(df_codificacion)

# guardar como csv el archivo csvTopicoArgendata_encoding_and_delimiters
# path_df_encoding_delimiters = '/home/lucia/Desktop/Fundar/Argendata_renaming/data/'
# df_resultado.to_csv(path_df_encoding_delimiters + 'csvTopicosArgendata_codificacion_y_delimitadores.csv', index = False)


     TOPICO                                       archivo_csv encoding  \
0    SALING                      ISA_composicion-ipcf_it4.csv    ascii   
1    SALING                            ISA_tipo_empleo_i1.csv    utf-8   
2    SALING                        ISA_salarios_region_i2.csv    ascii   
3    SALING                           ISA_salario_real_i1.csv    ascii   
4    SALING                                  ISA_ipcf_it1.csv    ascii   
..      ...                                               ...      ...   
332  MERTRA                  tasa_actividad_por_pais_anio.csv    utf-8   
333  MERTRA                     tasa_participacion_censos.csv    ascii   
334  MERTRA     tiempo_social_trabajo_sexo_niveleducativo.csv    utf-8   
335  MERTRA  tasa_empleo_por_franja_etaria_anio_provincia.csv    utf-8   
336  MERTRA                      tipo_trabajo_no_rem_sexo.csv    utf-8   

    delimiter  
0           ,  
1           ,  
2           ,  
3           ,  
4           ,  
..        ...  

<font color = "gold">  Ahora que tengo 'csvTopicosArgendata_codificacion_y_delimitadores.csv, voy a enfocarme en aquellas columnas que tengan datos de georreferencia </font> 

---

<font color = "gold"> De los .csv de todos los tópicos de Argendata/data voy a enfocarme en aquellos que tengan información de georreferencia. Para eso voy a usar dos inputs dados y algunas consideraciones: </font> 
- me pasaron un .txt con columnas que hasta el momento se sabe que son de georef -> georef_keys.txt
- voy a usar el geonomeclador de argendata, la pestaña curada export (geonomeclador_export.csv)
- de las columnas a investigar y buscar si corresponden a georeferencia, voy a quedarme con aquellas que: el 80% de su contenido aparezca en cada csv que busco

Puede que pase (a investigar en los siguientes pasos):
- aquellas columnas, de sus respectivos .csv y tópicos que no estén en español
- aquellos nombres/códigos que deberían ser cierta "nombre/codigo" y no lo son

---

<font color = "gold"> Primero trabajo con el archivo 'georef_keys.txt': </font> 

In [12]:
# primero trabajemos con el archivo 'georef_keys.txt':

ruta_txt = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/input/georef_keys.txt' #'/home/lucia/Desktop/Fundar/Argendata_renaming/data/input/georef_keys.txt' # 

# genero la lista con columnas que son de interés según 'georef_keys.txt'
input_TituloColumnas_geoRef = []

# abro el .txt y leo línea por línea
with open(ruta_txt, 'r', encoding = 'utf-8') as archivo:
    for linea in archivo:
        linea_limpia = linea.strip()  # acá saco espacios y saltos de línea, clave
        if linea_limpia != '':
            input_TituloColumnas_geoRef.append(linea_limpia)

print(input_TituloColumnas_geoRef)

# ahora voy a buscar en cuáles .csv de argendata aparecen los nombres que figuran en input_TituloColumnas_geoRef (georef_keys.txt)

coincidencias = []

# recorro y abro los csv segun la info de df_codificacion
for i in range(len(df_codificacion)):
    topico = df_codificacion.loc[i, 'TOPICO']
    archivo_csv = df_codificacion.loc[i, 'archivo_csv']
    encoding = df_codificacion.loc[i, 'encoding']
    delimiter = df_codificacion.loc[i, 'delimiter']

    ruta_csv = os.path.join(base_dir, topico, archivo_csv)

    try:
        df = pd.read_csv(ruta_csv, encoding = encoding, delimiter = delimiter)
    except Exception as e:
        print(f'Error leyendo {ruta_csv}: {e}')
        continue

    # verifico si alguna columna coincide con algun elemento de input_TituloColumnas_geoRef (georef_keys.txt)
    for columna in df.columns:
        if columna in input_TituloColumnas_geoRef:
            coincidencias.append({
                'TOPICO': topico,
                'archivo_csv': archivo_csv,
                'columna_georeferrencia': columna
            })

# crear df con las coincidencias encontradas
df_coincidencias_geoRefKeys = pd.DataFrame(coincidencias)

df_coincidencias_geoRefKeys

# por si necesitamos guardar en .csv es: TOPICO |archivo_csv | columna_georeferrencia
# df_coincidencias.to_csv('coincidencias_georef.csv', index=False)


['country_name_abbreviation', 'pais', 'partner_name_short_en', 'region', 'iso3_desc', 'partner_region_name_short_es', 'region_code', 'partner_region_id', 'pais_o_region', 'pais_o_grupo_de_paises', 'iso3', 'codigo_pais', 'country_code', 'partner_code', 'iso3c', 'countryname', 'nombre_pais', 'region_name', 'iso3_desc_fundar', 'pais_nombre']


,TOPICO,archivo_csv,columna_georeferrencia
0,SALING,ISA_regiones-ipcf_it1.csv,region
1,SALING,salario_real_ppa2017_ceped.csv,iso3
2,SALING,ISA_salarios_mundo_i1.csv,pais
3,SALING,ISA_salarios_region_i1.csv,region
4,SALING,ISA_regiones-ipcf_it2.csv,region
...,...,...,...
201,MERTRA,ratio_tasa_actividad_mujer_varon_por_pais_anio...,iso3
202,MERTRA,ratio_tasa_actividad_mujer_varon_por_pais_anio...,iso3_desc
203,MERTRA,puestos_percapita_anio_pais.csv,iso3
204,MERTRA,tasa_actividad_por_pais_anio.csv,iso3


<font color = "lightgreen"> El total de los csv en argendata con los que se arranca esta implementación tiene 337 .csv, de ese total 206 .csv (como mucho) tienen columnas georeferencia. La aclaración <i>como mucho</i> viene porque un mismo csv podría tener más de una columna asociada a información georreferencia. </font> 

----

### <font color = "gold"> Ahora trabajo con el geonomenglador dado, de la siguiente manera: </font> 

1. Para cada .csv Argendata (ubicado en la carpeta correspondiente de acuerdo al TOPICO), se abre  (con sus encoding y delimiter específicos) y se recorre cada una de sus columnas.

2. Para cada columna del .cvs Argendata, compara, por cada columna del geonomenclador, si al menos el 80% de los valores (después de eliminar los nulos) están contenidos en el conjunto de valores de la columna del geonomenclador.

3. Si se cumple la condición para alguna columna del geonomenclador, se registra una fila en el df_output con las columnas:

    • TOPICO

    • archivo_csv argendata
    
    • columnaGeorreferencia_80percGeoNom (el nombre de la columna del .csv argendata que cumple la condición)

In [13]:
# path y carga del geonomenclador de interés
geonomenclador_path = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/input/geonomenclador_export.csv'#'/home/lucia/Desktop/Fundar/Argendata_renaming/data/input/geonomenclador_export.csv'  # 
df_geonomenclador = pd.read_csv(geonomenclador_path)

df_geonomenclador

,geocodigo,name_long,name_short,iso_2
0,ABW,Aruba,Aruba,AW
1,AFE,África Oriental y Meridional,África Oriental y Meridional,NaN
2,AFG,Afganistán,Afganistán,AF
3,AFW,África Occidental y Central,África Occidental y Central,NaN
4,AGO,Angola,Angola,AO
...,...,...,...,...
1019,DESHUM_AHDI.WOF,Ramificaciones de Occidente,Ramificaciones de Occidente,NaN
1020,NIR,Irlanda del Norte,Irlanda del Norte,NaN
1021,CAMCLI_IA,Aviación internacional,Aviación internacional,NaN
1022,F_ESTPRO,África,África,NaN


#### <font color = "tomato"> Extraigo la columna, elimino nulos, normalizo: ESPACIÓN, MAYUSCULAS, TILDES </font> 
#### <font color = "tomato"> para que casos como los siguientes no rompan el 80%:</font> 
#### <font color = "tomato"> esto para que "NEA " y "NEA" o "pampeana" y "Pampeana" se reconozcan como idénticos</font> 
#### <font color = "tomato"> "AFGANISÁN" y "AFGANISTAN" (sin tilde) coincidirán</font> 

In [14]:
import unicodedata

# ruta de tabajo donde estan los topicos con sus csv 
base_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata' #'/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata' # 

# función para eliminar tildes
def sin_tildes(s):
    nkfd_form = unicodedata.normalize('NFKD', s)
    return "".join([c for c in nkfd_form if not unicodedata.combining(c)])

# lista para guardar los resultados
output_data = []

# ESTO ES POR CADA .csv DE LOS TOPICOS
for i in range(len(df_codificacion)):
    topico     = df_codificacion.loc[i, 'TOPICO']
    archivo_csv = df_codificacion.loc[i, 'archivo_csv']
    encoding   = df_codificacion.loc[i, 'encoding']
    delimiter  = df_codificacion.loc[i, 'delimiter']
    
    # ruta completa de cada archivo .csv
    path_csv = os.path.join(base_dir, topico, archivo_csv)
    
    # leer cada csv utilizando el encoding y delimiter especificados
    try:
        df_csv_argendata = pd.read_csv(path_csv, encoding=encoding, delimiter=delimiter)
    except Exception as e:
        print('Error leyendo el archivo:', path_csv, 'Error:', e)
        continue  # si falla un csv, me dice cuál y sigue con el siguiente
    
    # recorrer cada columna del csv leído, esto es por csv
    for col_csv in df_csv_argendata.columns:
        # extraigo la columna, elimino nulos, normalizo y quito tildes
        # para que casos como los siguientes no rompan el 80%:
        # esto para que "NEA " y "NEA" o "pampeana" y "Pampeana" se reconozcan como idénticos
        # "AFGANISTAN" y "AFGANISTAN" (sin tilde) coincidirán
        serie = (
            df_csv_argendata[col_csv]
            .dropna()
            .astype(str)
            .str.strip()
            .str.upper()
            .apply(sin_tildes)
        )
        total_count = len(serie)
        
        # si la columna está vacía, la salto
        if total_count == 0:
            continue

        # recorro cada columna del geonomenclador
        for col_geo in df_geonomenclador.columns:
            # preparo el set de comparación: sin nulos, string, strip, upper, sin tildes
            geo_set = set(
                df_geonomenclador[col_geo]
                .dropna()
                .astype(str)
                .str.strip()
                .str.upper()
                .apply(sin_tildes)
            )
            
            # cuento cuántos valores de serie están en geo_set
            match_count = sum(1 for v in serie if v in geo_set)
            
            # calculo el porcentaje de coincidencia
            porcentaje = match_count / total_count
            
            # si hay al menos 80%, registro col_csv y paso a la siguiente columna del CSV
            if porcentaje >= 0.8:
                output_data.append({
                    'TOPICO': topico,
                    'archivo_csv': archivo_csv,
                    'columnaGeorreferencia_80percGeoNom': col_csv
                })
                # rompo este bucle de geonomenclador para no repetir la misma col_csv
                break  

# df final con todas las columnas de cada CSV que cumplen ≥80% de coincidencia
df_col_80percGeoNom = pd.DataFrame(output_data)

df_col_80percGeoNom

# guardar el resultado en un archivo csv
# df_col_80percGeoNom.to_csv('columnscsv_80percGeoNom.csv', index=False)


,TOPICO,archivo_csv,columnaGeorreferencia_80percGeoNom
0,SALING,ISA_salarios_region_i2.csv,variable
1,SALING,ISA_regiones-ipcf_it1.csv,region
2,SALING,salario_real_ppa2017_ceped.csv,iso3
3,SALING,ISA_salarios_mundo_i1.csv,pais
4,SALING,ISA_salarios_region_i1.csv,region
...,...,...,...
215,MERTRA,puestos_percapita_anio_pais.csv,iso3
216,MERTRA,tasa_empleo_provincia_nivel_educativo.csv,provincia
217,MERTRA,tasa_actividad_por_pais_anio.csv,iso3
218,MERTRA,tasa_actividad_por_pais_anio.csv,iso3_desc


---

### <font color = "gold"> ¿Qué le faltaría al geonomenclador_Argendata (en principio) que sí tiene georef_keys.txt? y viceversa </font> 

Ahora que tengo df_col_80percGeoNom, que tiene como columnas TOPICO | archivo_csv | columnaGeorreferencia_80percGeoNom; y el dataframe df_coincidencias_geoRefKeys que tiene las columnas TOPICO | archivo_csv | columna_georeferrencia armo dos nuevos dfs. Uno que sea GeoNomYes_GeoKeysNo donde se guarde como TOPICO | archivo_csv | GeoNomYes_GeoKeysNo cuales son las filas del dataframe df_col_80percGeoNom que no aparecen en el dataframe df_coincidencias_geoRefKeys. De manera análoga, el otro dataframe que quiero generar es GeoKeysYes_GeoNomNo que guarde como TOPICO | archivo_csv | GeoKeysYes_GeoNomNo cuales son las filas del dataframe df_coincidencias_geoRefKeys que no aparecen en el dataframe df_col_80percGeoNom


In [15]:
# quisiste no usar merge y te la pusiste con tuplas, no lo vuelvas a intentar

# para hacerme más fácil la evaluación y comparación entre dfs, solo para este paso voy a renombrar cositas
df_geo_nom = df_col_80percGeoNom.rename(columns={'columnaGeorreferencia_80percGeoNom': 'geo'})
df_geo_keys = df_coincidencias_geoRefKeys.rename(columns={'columna_georeferrencia': 'geo'})

# acá voy a documentar en detalle por mi propio bien:
# vamos a usar merge para combinar df_geo_nom y df_geo_keys utilizando una 'conjunción' basada en las columnas 'TOPICO', 'archivo_csv' y 'geo'. 

# parámetro how = 'left' (que es el que voy a usar para mergear) asegura que se mantengan todas las filas de df_geo_nom y se traigan las filas que coincidan de df_geo_keys
# parámetro indicator = True agrega una columna extra llamada '_merge', que nos indica el origen de cada fila resultante:
#   - 'both': la fila existe en ambos df.
#   - 'left_only': la fila existe únicamente en el DataFrame de la izquierda (df_geo_nom) 
#   - 'right_only': la fila existe únicamente en el DataFrame de la derecha.
merge_nom = df_geo_nom.merge(df_geo_keys, on=['TOPICO', 'archivo_csv', 'geo'],
                             how = 'left', indicator = True)
#print(merge_nom)
# vivan merge y sus parametros :>

# ahora, con la super columna _merge filtro únicamente las filas que están presentes solo en df_geo_nom (la condición _merge == 'left_only')
# con esto, logramos quedarnos con (TOPICO, archivo_csv, geo) que está presente en df_geo_nom pero NO en df_geo_keys
df_GeoNomYes_GeoKeysNo = merge_nom[merge_nom['_merge'] == 'left_only'][['TOPICO', 'archivo_csv', 'geo']]

# 'geo' a 'GeoNomYes_GeoKeysNo' para dejar claro el resultado
df_GeoNomYes_GeoKeysNo = df_GeoNomYes_GeoKeysNo.rename(columns={'geo': 'GeoNomYes_GeoKeysNo'})

df_GeoNomYes_GeoKeysNo = df_GeoNomYes_GeoKeysNo.reset_index(drop=True)

print('geoKeys.txt:', input_TituloColumnas_geoRef)
df_GeoNomYes_GeoKeysNo


geoKeys.txt: ['country_name_abbreviation', 'pais', 'partner_name_short_en', 'region', 'iso3_desc', 'partner_region_name_short_es', 'region_code', 'partner_region_id', 'pais_o_region', 'pais_o_grupo_de_paises', 'iso3', 'codigo_pais', 'country_code', 'partner_code', 'iso3c', 'countryname', 'nombre_pais', 'region_name', 'iso3_desc_fundar', 'pais_nombre']


,TOPICO,archivo_csv,GeoNomYes_GeoKeysNo
0,SALING,ISA_salarios_region_i2.csv,variable
1,INFDES,tasa_informalidad_provincia_definicion_product...,prov_desc
2,INFDES,satisfaccion_vida_desempleo_pais.csv,pais_desc
3,INFDES,tasa_desempleo_sexo_paises_modelada.csv,pais_desc
4,INFDES,tasa_desempleo_arg_mundial_modelada.csv,pais_desc
5,ESTPRO,densidad_empresarial_depto.csv,provincia
6,ESTPRO,densidad_nbi.csv,provincia
7,ESTPRO,vab_sectorial_provincia.csv,provincia
8,CRECIM,provincia_part_vab.csv,provincia_id
9,CRECIM,vab_por_provincia.csv,provincia_id


In [16]:
# pasemos ahora al otro caso, identificar las filas que están en df_geo_keys pero no en df_geo_nom -> realizamos el proceso inverso
# ie, usamos df_geo_keys como df principal y hacemos un left merge con df_geo_nom
# así, obtenemos filas con _merge == 'left_only' que indican que esa combinación solo aparece en df_geo_keys

merge_keys = df_geo_keys.merge(df_geo_nom, on=['TOPICO', 'archivo_csv', 'geo'],
                               how = 'left', indicator = True)

# nos quedamos las filas que están solo en df_geo_keys
df_GeoKeysYes_GeoNomNo = merge_keys[merge_keys['_merge'] == 'left_only'][['TOPICO', 'archivo_csv', 'geo']]

# la columna 'geo' a 'GeoKeysYes_GeoNomNo' para diferenciar
df_GeoKeysYes_GeoNomNo = df_GeoKeysYes_GeoNomNo.rename(columns={'geo': 'GeoKeysYes_GeoNomNo'})

df_GeoKeysYes_GeoNomNo = df_GeoKeysYes_GeoNomNo.reset_index(drop=True)
df_GeoKeysYes_GeoNomNo

,TOPICO,archivo_csv,GeoKeysYes_GeoNomNo
0,PRECIO,7_comparacion_inflacion_mediana_argentina_lata...,pais_o_grupo_de_paises
1,ACECON,9_pibpc_ppa_log_1950.csv,pais_o_region
2,CRECIM,vab_por_provincia.csv,region
3,DESIGU,ISA_mundo_i1.csv,pais
4,DESIGU,ISA_mundo_i2.csv,pais
5,DESIGU,ISA_movilidad_i2.csv,pais
6,POBREZ,ISA_pobreza_monetaria_it2.csv,region
7,POBREZ,ISA_global-poverty_it3.csv,region_name
8,COMEXT,composicion_importaciones_bienes_sectores_Bram...,country_name_abbreviation
9,COMEXT,cambio_destinos_exportacion.csv,partner_name_short_en


#### <font color = "tomato"> En el df df_GeoKeysYes_GeoNomNo me gustaría que haya, hasta el momento, solo NO detección de columnas en idiomas distintos al español pero hay otros problemas (excel) </font>

#### <font color = "tomato"> Los registro en archivos Argendata_problemaRenaming.ods, guardado en mi repo  </font>

In [57]:
tx = pd.read_csv('/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata/TRANEN/potencia_instalada_renov_regional.csv')
tx

valores_unicos = tx['region'].dropna().unique().tolist()
print(valores_unicos)

['CABA y Provincia de Buenos Aires', 'Centro', 'Comahue', 'Cuyo', 'Litoral', 'Noreste Argentino', 'Noroeste Argentino', 'Patagonia', 'Total']


In [39]:
df_geonomenclador

,geocodigo,name_long,name_short,iso_2
0,ABW,Aruba,Aruba,AW
1,AFE,África Oriental y Meridional,África Oriental y Meridional,NaN
2,AFG,Afganistán,Afganistán,AF
3,AFW,África Occidental y Central,África Occidental y Central,NaN
4,AGO,Angola,Angola,AO
...,...,...,...,...
1019,DESHUM_AHDI.WOF,Ramificaciones de Occidente,Ramificaciones de Occidente,NaN
1020,NIR,Irlanda del Norte,Irlanda del Norte,NaN
1021,CAMCLI_IA,Aviación internacional,Aviación internacional,NaN
1022,F_ESTPRO,África,África,NaN


#### <font color = "gold"> ¿Por qué hice esto?: posible estrategia para refinar y enriquecer un input (un .txt) que ayude a detectar columnas con información georreferencial de un csv </font>

In [23]:
# valores únicos de la columna GeoNomYes_GeoKeysNo.
nuevos_valores = df_GeoNomYes_GeoKeysNo['GeoNomYes_GeoKeysNo'].unique().tolist()

new_GeoRefKeys = input_TituloColumnas_geoRef.copy()

# por si hay duplicados, vamos a iterar sobre estos valores y añadirlos a input_TituloColumnas_geoRef solo si no están presentes
for valor in nuevos_valores:
    if valor not in new_GeoRefKeys:
        new_GeoRefKeys.append(valor)

print(input_TituloColumnas_geoRef)
print(new_GeoRefKeys)


['country_name_abbreviation', 'pais', 'partner_name_short_en', 'region', 'iso3_desc', 'partner_region_name_short_es', 'region_code', 'partner_region_id', 'pais_o_region', 'pais_o_grupo_de_paises', 'iso3', 'codigo_pais', 'country_code', 'partner_code', 'iso3c', 'countryname', 'nombre_pais', 'region_name', 'iso3_desc_fundar', 'pais_nombre']
['country_name_abbreviation', 'pais', 'partner_name_short_en', 'region', 'iso3_desc', 'partner_region_name_short_es', 'region_code', 'partner_region_id', 'pais_o_region', 'pais_o_grupo_de_paises', 'iso3', 'codigo_pais', 'country_code', 'partner_code', 'iso3c', 'countryname', 'nombre_pais', 'region_name', 'iso3_desc_fundar', 'pais_nombre', 'provincia', 'prov_desc', 'pais_desc', 'continente_fundar', 'provincia_desc', 'code', 'variable', 'provincia_id', 'descripcionpais', 'nom_pcia']


### <font color = "tomato"> Se obtiene lo que muestro abajo. Un "problemita" de esto es que puede haber columnas que contienen información georreferencial pero que pueden llamarse de alguna manera "poco representativa", como por ejemplo <i> variable </i> </font>.
Este es el caso de ISA_salarios_region_i2.csv

### <font color = "tomato"> Entonces, el siguiente input sería útil, pero no suficiente: </i> </font>

<b> new_GeoRefKeys: </b>
['country_name_abbreviation', 'pais', 'partner_name_short_en', 'region', 'iso3_desc', 'partner_region_name_short_es', 'region_code', 'partner_region_id', 'pais_o_region', 'pais_o_grupo_de_paises', 'iso3', 'codigo_pais', 'country_code', 'partner_code', 'iso3c', 'countryname', 'nombre_pais', 'region_name', 'iso3_desc_fundar', 'pais_nombre', 'variable', 'prov_desc', 'pais_desc', 'provincia', 'provincia_id', 'code', 'descripcionpais', 'continente_fundar', 'nom_pcia', 'provincia_desc']


### <font color = "tomato"> Pues no toda columna que se llame <i> variable </i> será georreferencia, sentido común. </i> </font>


In [24]:
df_test_columnaPais =  pd.read_csv('/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata/ACECON/A1_inb_pib.csv') # pd.read_csv('/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata/ACECON/A1_inb_pib.csv')
print(df_test_columnaPais)

df_col_80percGeoNom

      anio        pais iso3  diferencia_inb_pbi
0     1960  Afganistan  AFG            2.066102
1     1961  Afganistan  AFG            2.024294
2     1962  Afganistan  AFG            2.032524
3     1963  Afganistan  AFG            2.071000
4     1964  Afganistan  AFG            1.944440
...    ...         ...  ...                 ...
8941  2018    Zimbabwe  ZWE           -2.087977
8942  2019    Zimbabwe  ZWE           -1.577477
8943  2020    Zimbabwe  ZWE           -2.222734
8944  2021    Zimbabwe  ZWE           -2.463721
8945  2022    Zimbabwe  ZWE           -2.791389

[8946 rows x 4 columns]


,TOPICO,archivo_csv,columnaGeorreferencia_80percGeoNom
0,SEBACO,22_participacion_expo_sbc.csv,iso3
1,SEBACO,25_impo_x_socio_comercial_arg.csv,iso3
2,SEBACO,23_participacion_expo_ssi.csv,iso3
3,SEBACO,06_empleo_sbc_provincia.csv,provincia
4,SEBACO,24_expo_x_socio_comercial_arg.csv,iso3
...,...,...,...
215,INFDES,composicion_empleo_ultimo_anio_latam.csv,pais
216,INFDES,anios_educacion_genero_por_pais.csv,iso3
217,INFDES,anios_educacion_genero_por_pais.csv,pais
218,MINERI,minerales_vbp_mundial.csv,codigo_pais



'/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata/ACECON/A1_inb_pib.csv' contiene la columna pais pero la condición 80% no la leía pues:

```
total = len(serie)                      # 8946 filas no nulas  
matches = 6111                          # sólo esas coincidencias exactas  
ratio = matches / total  ≈ 0.683        # 68.3 % < 80 %
```

<font color = "gold"> RESUELTO METIÉNDOME CON HOMOGENEIZAR TEMAS TILDES, TEMA ESPACIOS COMO "NEA "/ "NEA", Y MAYÚSCULAS </font> 

In [26]:
listaColumnas_GeoKeysYes_GeoNomNo = df_GeoKeysYes_GeoNomNo['GeoKeysYes_GeoNomNo'].unique().tolist()
print(listaColumnas_GeoKeysYes_GeoNomNo)

['iso3_desc', 'pais', 'region', 'countryname', 'country_name_abbreviation', 'partner_name_short_en', 'partner_region_id', 'partner_region_name_short_es', 'pais_o_grupo_de_paises', 'region_name', 'pais_o_region']


#  <u> <font color = "orangered"> En cuanto a implementar el proceso de renombre: </font> </u>

In [ ]:
# ruta original de topicos argendata con sus csv
src_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata'
# destino se va a guardar la nueva estructura con csv modificados y/o copiados sin cambio
dest_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata_new'

#%% PREPARO LA ESTRUCTURA DEL DESTINO FINAL

# copio la estructura de las carpetas (tópicos) de la ruta original
topics_all = []
for item in os.listdir(src_dir):
    item_path = os.path.join(src_dir, item)
    if os.path.isdir(item_path) and item not in ['LICENSE', 'README.md']:
        topics_all.append(item)

# primeros 3 topicos para la prueba
topics_to_process = sorted(topics_all)[:3] #orden alfabetico para comparar "mas facil" mientras veo los topicos y archivos
print('Tópicos a procesar (prueba):', topics_to_process)

# voy a trabajar las partes claves, para mi, del geonomenclador como un diccionario, por facilidad
# la columna 'geocodigo' como key y 'name_long' su valor
geo_busqueda = {}
for idx in range(len(df_geonomenclador)):
    key_val = str(df_geonomenclador.loc[idx, 'geocodigo']).strip()    # evitar problemitas como: "AFW" y "AFW "
    name_val = df_geonomenclador.loc[idx, 'name_long']
    geo_busqueda[key_val] = (key_val, name_val)

#%% PASEO POR CADA TOPICO Y SUS .csv

for topic in topics_to_process:
    src_topic_dir = os.path.join(src_dir, topic)        
    dest_topic_dir = os.path.join(dest_dir, topic)

    # crear la carpeta de destino en base a la estructura de la de origen
    if not os.path.exists(dest_topic_dir):
        os.makedirs(dest_topic_dir)

    # listita de los archivos del tópico
    csv_files = [] 
    for f in os.listdir(src_topic_dir):
        if f.endswith('.csv'):
            csv_files.append(f)

    for csv_file in csv_files:

        #%% BUSCAR EN df_codificacion la fila que tenga TOPICO == topic y archivo_csv == csv_file del .csv de interés

        encoding = None
        delimiter = None
        for j in range(len(df_codificacion)):
            if df_codificacion.loc[j, 'TOPICO'] == topic and df_codificacion.loc[j, 'archivo_csv'] == csv_file:
                encoding = df_codificacion.loc[j, 'encoding']
                delimiter = df_codificacion.loc[j, 'delimiter']
                break # corto cuando encuentra el encoding y delimiter del .csv 
        if encoding is None or delimiter is None:
            #esto por si alguien después lo corre sin actualizar el df_codificacion
            print(f'Información de encoding-delimiter no encontrada para {csv_file} en tópico {topic}. Se omite archivo.') 
            continue

        src_csv_path = os.path.join(src_topic_dir, csv_file)
        try:
            df_csv = pd.read_csv(src_csv_path, encoding = encoding, delimiter = delimiter)
        except Exception as e:
            print(f'Error al leer {src_csv_path}: {e}') #esto por si alguien después lo corre sin actualizar el df_codificacion
            continue

        #%% EXPLORAR Y VERIFICAR LA CONDICIÓN DEL 80% DE INFORMACIÓN GEORREFERENCIAL, SEGÚN EL GEONOMENCLADOR
        candidate_col = None  # si una columna cumple la condición la guardo acá
        for col in df_csv.columns:
            series_csv = df_csv[col].dropna().astype(str) # columna a string y eliminar valores nulos
            if len(series_csv) == 0:
                continue  # si la columna queda vacía, seguimos
            total_count = len(series_csv)
            qualifies = False  # como hice antes, lo uso para indicar si esta columna cumple la condición

            # para cada columna del geonomenclador, comprobamos si al menos el 80% de los valores aparece
            for geo_col in df_geonomenclador.columns:
                set_geo = set(df_geonomenclador[geo_col].dropna().astype(str)) # uso set por lo que ya conté antes
                match_count = 0
                for valor in series_csv:
                    if valor in set_geo:
                        match_count += 1
                if match_count / total_count >= 0.8:
                    qualifies = True
                    break  # se encontró que esta columna del .csv cumple la condición con alguna columna del geonomenclador

            if qualifies:
                candidate_col = col
                break  # estoy tomando la primera columna que cumple la condición, aclaro por las observaciones de antes

        #%% MODIFICAMOS EL .csv O LO COPIAMOS
        if candidate_col is not None:
            # si encontramos columna candidata, se agregan las columnas codigo_geo y nombre_largo (era para no marearme con los del geonomenclador_export)
            nuevo_codigo_geo = []
            nuevo_nombre_largo = []
            for idx in range(len(df_csv)):
                # tomamos el valor en la columna candidata y lo convertimos a string
                value = str(df_csv.loc[idx, candidate_col]).strip()
                # hacemos la exploracion en el geo_busqueda -> si no se encuentra la key, queda None
                if value in geo_busqueda:
                    nuevo_codigo_geo.append(geo_busqueda[value][0])
                    nuevo_nombre_largo.append(geo_busqueda[value][1])
                else:
                    nuevo_codigo_geo.append(None)
                    nuevo_nombre_largo.append(None)
            # agregamos las dos columnas nuevas al df del correspondiente .csv
            df_csv['codigo_geo'] = nuevo_codigo_geo
            df_csv['nombre_largo'] = nuevo_nombre_largo
            print(f'{csv_file} en {topic} es MODIFICADO a partir de la columna candidata {candidate_col}.')
        else:
            print(f'{csv_file} en {topic} NO cumple con la condición georreferencial y se copia sin cambios.')

        #%% GUARDAMOS EL .csv MODIFICADO (O SIN CAMBION) EN EL NUEVO ESQUEMITA
        dest_csv_path = os.path.join(dest_topic_dir, csv_file)
        try: # así tomo nota de con que .csv tal vez tengo problema
            df_csv.to_csv(dest_csv_path, index = False, encoding = encoding) 
            # index = False para no escribirle al .csv la columna de índices 
            #  encoding = encoding utiliza la misma codificación que especificamos para leer el archivo originalmente, mantener consistencia
            print(f'Guardado en {dest_csv_path}.')
        except Exception as e:
            print(f'\nError al guardar {dest_csv_path}: {e}')

Tópicos a procesar (prueba): ['ACECON', 'AGROPE', 'CAMCLI']
A5_pibpc_propeeuu_ppa_usd.csv en ACECON es MODIFICADO a partir de la columna candidata iso3.

Error al guardar /home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata_new/ACECON/A5_pibpc_propeeuu_ppa_usd.csv: 'ascii' codec can't encode character '\xe1' in position 45: ordinal not in range(128)
8_pib_dist.csv en ACECON NO cumple con la condición georreferencial y se copia sin cambios.
Guardado en /home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata_new/ACECON/8_pib_dist.csv.
2_pibpc_salud_edu.csv en ACECON es MODIFICADO a partir de la columna candidata iso3.

Error al guardar /home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata_new/ACECON/2_pibpc_salud_edu.csv: 'ascii' codec can't encode character '\xed' in position 39: ordinal not in range(128)
A3_ipc_ipi.csv en ACECON NO cumple con la condición georreferencial y se copia sin cambios.
Guardado en /home/capucc

### <font color = "tomato"> Renaming para los 3 primeros tópicos en orden ACECON, AGROPE, CAMCLI: </font>

- v1 del renaming: estoy usando solo la información del geonomenclador

- la estructura de la nueva carpetam sale joya

- mucho caso de éxito (?). Por ejemplo: 

produccion_global_carne_aviar_2021.csv en AGROPE es MODIFICADO a partir de la columna candidata iso3.
Guardado en /home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata_new/AGROPE/produccion_global_carne_aviar_2021.csv.

- Hay casos que parecen de éxito pero no lo son, como es el siguiente caso. Se detecta bien que hay (por lo menos una) columna de georreferencia, pero como me enfoqué en un diccionario de busqueda con 'geocodigo' como key 'name_long' su valor, me quede corta en casos como provincias. Ya sé (creo?) como arreglarlo. El ejemplo:  

PBG_participacion_agro_provincia_evolucion.csv en AGROPE es MODIFICADO a partir de la columna candidata nom_pcia.
Guardado en /home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata_new/AGROPE/PBG_participacion_agro_provincia_evolucion.csv.

- en algunos casos tengo un tema de encoding al guardar. Se evalúa para v2. Por ejemplo: 

A4_efecto_bs.csv en ACECON es MODIFICADO a partir de la columna candidata iso3.
Error al guardar /home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata_new/ACECON/A4_efecto_bs.csv: 'ascii' codec can't encode character '\xe1' in position 43: ordinal not in range(128)  

- tengo varios casos puntuales que quiero ver con detalle + incorporar idioma + input extra


### <font color = "tomato">  Volver a ver tema ESPACIOS (por si están en el medio)</font>

### <font color = "tomato">  Quiero ver lo de idioma </font>